In [8]:
# Install necessary libraries
!pip install sentence-transformers faiss-cpu transformers


In [11]:
from google.colab import files
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer


# Pre-Processinng Slop

In [12]:


def read_txt(file_path):
    with open(file_path, 'r') as file:
        return file.read()


inquisitor_lore_text = read_txt('Inquisitor Lore.txt')
promptset_text = read_txt('promptset.txt')

# Combine documents into a list
documents = [inquisitor_lore_text, promptset_text]


In [13]:

embedder = SentenceTransformer('all-MiniLM-L6-v2')

doc_embeddings = embedder.encode(documents, show_progress_bar=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# Build FAISS index for fast retrieval

In [14]:
# Step 4: Build FAISS index for fast retrieval

# Get the dimensionality of the embeddings
dimension = doc_embeddings.shape[1]

# Initialize the FAISS index with L2 distance metric
index = faiss.IndexFlatL2(dimension)

# Add the document embeddings to the index
index.add(np.array(doc_embeddings))


# Building The Retriever

In [15]:


def retrieve_relevant_docs(query, top_k=2):

    query_embedding = embedder.encode([query])


    distances, indices = index.search(np.array(query_embedding), top_k)


    return [documents[i] for i in indices[0]]


# Building The Generator using T5

In [16]:


t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')

def generate_answer(query, retrieved_docs):

    context = " ".join(retrieved_docs)
    input_text = f"Question: {query} Context: {context}"


    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")


    output_ids = t5_model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)


    answer = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer


In [ ]:


# Sample Query
query = "What is the goal of this project?"

# Retrieve the most relevant documents based on the query
retrieved_docs = retrieve_relevant_docs(query)

# Generate the answer using the retrieved documents
answer = generate_answer(query, retrieved_docs)

# Print the generated answer
print("Generated Answer:", answer)


Token indices sequence length is longer than the specified maximum sequence length for this model (11210 > 512). Running this sequence through the model will result in indexing errors
